## Genome informatics
# RNA-Seq analysis : Quantification

## Motivation for RNA-Seq quantification

- Typical studies:
    - DNA-Seq -> alignment -> variant calling -> genome wide association study (GWAS);
  
    <img src="images/GWAS2-700x438.png" width="400">
    
    - RNA-Seq -> alignment -> quantification -> differential expression analysis

## RNA-Seq quantification

Raw counting or probabilistic estimation dilemma is (mostly) about doing analysis on gene vs transcript level.
- When quantifying on **gene** level - we simply count the number of reads aligned to each gene.
- On **transcript** level, we need an algorithm such as the Expectation Maximization (EM, pictured below) to deal with the uncertainty.

<br>
<img src="images/em1.png" width="550">

<img src="images/em2.png" width="550">

When performing statistical analysis of RNA expression, doing it on gene level compared to transcript level is more robust and experimentally actionable. The biologists will also usually draw their conclusions on gene level since that's the level that the biological pathways are annotated on. 

However, the use of gene counts for statistical analysis can mask transcript-level dynamics. A popular alternative nowadays is to estimate the transcript abundances and then aggregate to gene level, or perform the entire analysis on trancsript level (testing for differential expression) and then aggregate the results.

## Gene level quantification

We shall, for simplicity's sake, only perform gene level quantification. Even though it has some drawbacks, it's a strategy still used by most genomic scientists. Here's what we have at the beginning of the quantification step and what we want to estimate:

- We **have**: aligned reads and annotations. A file format called SAM/BAM is now the standard formats for next-generation sequence alignments.
- We **estimate**: relative abundance.

We'll start with some simple methods to find if two intervals overlap.

In [3]:
def overlap(x, y):
    """
    This function takes two intervals and determines whether 
    they have any overlapping segments.
    """
    new_start = max(x[1], y[1])
    new_end = min(x[2], y[2])
    
    if new_start < new_end and x[0] == y[0]:
        return True
    return False

We will define four intervals as tuples.

In [4]:
region1 = ("chr3", 27, 82)
region2 = ("chr4", 27, 82)
region3 = ("chr3", 57, 75)

In [5]:
overlap(region1, region2)

False

In [6]:
overlap(region1, region3)

True

We shall now pick a gene of interest and try to find reads mapping to that gene.

In [7]:
# the location of DEFB125 gene in human genome, gencode.v27 annotation
gene = ('chr20', 87249, 97094)

To start exploring reads from a SAM/BAM file, we first need to load the file.

In [10]:
import pysam

# load BAM file
bamfile = pysam.AlignmentFile("aligned/sample_01_accepted_hits.bam", "rb")

Note the 'rb' parameter indicates to read the file as binary, which is the BAM format (BAM stands for Binary Alignment Map). If loading a SAM file, this parameter does not need to be specified

An important and very common application is to count the number of reads (aligned fragments) that overlap a given feature (i.e. region of the genome or gene). One simple approach to doing this is to make a list of all reads generated and simply iterate over the reads to identify whether a read overlaps a region. 

In this case we need an overlap method that can compare a simple interval (defined by a tuple of sequence, start, end) with a pysam AlignedSegment object. This overlap method also needs the AlignmentFile object to decode the chromosome name.

In [8]:
def overlap(x, gene, bamfile):
    """
    A modified version of overlap that takes an interval and a pysam
    AlignedSegment and tests for overlap
    """
    new_start = max(x.reference_start, gene[1])
    new_end = min(x.reference_end, gene[2])
    
    if (new_start < new_end and bamfile.getrname(x.tid) == gene[0]):
        return True
    
    return False

In [11]:
%%time

# code to iterate over reads and count for a single gene
naive_gene_count = 0

global ref

for x in bamfile.fetch():
    # Note x is of type pysam.AlignedSegment
    if(overlap(x, gene, bamfile)):
        naive_gene_count += 1
    ref = x.reference_end
    

CPU times: user 3.98 s, sys: 34.9 ms, total: 4.01 s
Wall time: 4.02 s


In [12]:
print(naive_gene_count)

1016


The problem with this approach is that to do this, you will need to store the entire file in memory. Worse than that, in order to find the reads within a single gene, you would need to iterate over the entire file, which can contain hundreds of millions of reads. This is quite slow even for a single gene, but will only increase if you want to look at many genes.

Instead, more efficient datastructures (and indexing schemes) can be used to retrieve reads based on positions in more efficient ways.

To get the reads overlapping a given region, Pysam has a method called fetch(), which takes the genomic coordinates and returns an iterator of all reads overlapping that region.

In [13]:
bam_iter = bamfile.fetch(gene[0], gene[1], gene[2])

In [14]:
%%time

pysam_gene_count = 0

for x in bam_iter:
    pysam_gene_count += 1

CPU times: user 2.21 ms, sys: 878 µs, total: 3.08 ms
Wall time: 2.07 ms


In [15]:
print(pysam_gene_count)

1016


Not surprisingly, they are the same - the big difference is that the second method is significantly faster. The reason for this is that Pysam (like Samtools, Picard, and other similar toolkits) make use of clever tricks to index the file by genomic positions to more efficiently search for reads within a given genomic interval.

Now that we know how to count reads overlapping a region, we can write this as a function and try and compute this for all genes.

In [16]:
def read_count(gene, bamfile):
    """
    Compute the number of reads contained in a bamfile that overlap
    a given interval
    """
    bam_iter = bamfile.fetch(gene[0], gene[1], gene[2])

    pysam_gene_count = 0
    
    for x in bam_iter:
        pysam_gene_count += 1
        
    return pysam_gene_count

You can run this with any gene tuple now:

In [17]:
# the location of ZNFX1 gene in human genome, gencode.v27 annotation
gene2 = ("chr20", 49237945, 49278426)

read_count(gene2, bamfile)

5282

Now, let's compute the gene counts for all genes. We'll start by reading in all genes:

In [19]:
%%time

gene_counts={}

with open('gencode.v27.chr20.bed', 'r') as f:
    for line in f:
        tokens = line.split('\t')
        gene_local = (tokens[0], int(tokens[1]), int(tokens[2]))
        count = read_count(gene_local, bamfile)
        gene_counts.update({tokens[3].rstrip() : count})   

CPU times: user 4.73 s, sys: 72.8 ms, total: 4.81 s
Wall time: 4.81 s


Now it's easy to query the gene counts for different genes.

In [20]:
print(gene_counts.get("ARFGEF2"))
print(gene_counts.get("SOX12"))
print(gene_counts.get("WFDC8"))

7283
3836
1641
